In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import h5py as h5
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.cluster import KMeans
# Any results you write to the current directory are saved as output.

C:\Users\Andre\Anaconda3\envs\GreatEnv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
parent_cat_to_english = {
    'Личные вещи': 'personal things',
    'Для дома и дачи': 'For home and cottages',
    'Бытовая электроника': 'cellular electronics',
    'Транспорт': 'transport',
    'Недвижимость': 'the property',
    'Животные': 'animals',
    'Хобби и отдых': 'hobbies and recreation',
    'Услуги': 'the services', 
    'Для бизнеса': 'for business'
}

cats = {
    'Аудио и видео': 'audio and video',
    'Игры, приставки и программы': 'games, consoles and programs',
    'Настольные компьютеры': 'desktop pc',
    'Ноутбуки': 'laptop',
    'Оргтехника и расходники': 'office equipment and consumables',
    'Планшеты и электронные книги': 'tablets and ebooks',
    'Телефоны': 'phones',
    'Товары для компьютера': 'computer products',
    'Фототехника': 'photographic equipment',
    'Готовый бизнес': 'ready business',
    'Оборудование для бизнеса': 'equipment for business',
    'Бытовая техника': 'appliances',
    'Мебель и интерьер': 'furnature and interior',
    'Посуда и товары для кухни': 'dishes and products for the kitchen',
    'Продукты питания': 'food',
    'Растения': 'plants',
    'Ремонт и строительство': 'repair and construction',
    'Аквариум': 'aquarium',
    'Другие животные': 'other animals',
    'Кошки': 'cats',
    'Птицы': 'birds',
    'Собаки': 'dogs',
    'Товары для животных': 'goods for pets',
    'Детская одежда и обувь': 'childrens clothing and footwear',
    'Красота и здоровье': 'beauty and health',
    'Одежда, обувь, аксессуары': 'clothes shoes accessories',
    'Товары для детей и игрушки': 'goods for children and toys',
    'Часы и украшения': 'watches and jewelry',
    'Гаражи и машиноместа': 'garages and parking places',
    'Дома, дачи, коттеджи': 'houses and cottages',
    'Земельные участки': 'land',
    'Квартиры': 'apartment',
    'Коммерческая недвижимость': 'commercial property',
    'Комнаты': 'rooms',
    'Недвижимость за рубежом': 'propery abroad',
    'Автомобили': 'cars',
    'Водный транспорт': 'water transport',
    'Грузовики и спецтехника': 'trucks and special equipment',
    'Мотоциклы и мототехника': 'motorcycle and equipment',
    'Предложение услуг': 'offer of services',
    'Билеты и путешествия': 'tickets and travel',
    'Велосипеды': 'bicycles',
    'Книги и журналы': 'books and mags',
    'Коллекционирование': 'collecting',
    'Музыкальные инструменты': 'musical instruments',
    'Охота и рыбалка': 'hunting and fishing',
    'Спорт и отдых': 'sport and leisure',
}

In [3]:
trainDF = pd.read_csv("input/avito-demand-prediction/train.csv")
vggFeat = h5.File("AvitoVGGFeatures_Train.h5", 'r')
vggFeat = np.array(vggFeat["vgg"])

badImgs = ["4f029e2a00e892aa2cac27d98b52ef8b13d91471f613c8d3c38e3f29d4da0b0c",
           "b98b291bd04c3d92165ca515e00468fd9756af9a8f1df42505deed1dcfb5d7ae",
           "60d310a42e87cdf799afcd89dc1b11ae3fdc3d0233747ec7ef78d82c87002e83",
           "8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58"]

trainDF[trainDF["image"].isin(badImgs)] = np.nan
    
trainDF = trainDF[~trainDF["image"].isnull()][0:vggFeat.shape[0]]
trainDF["category_name"] = trainDF["category_name"].map(lambda x: cats[x])

trainDF.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,goods for children and toys,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2.0,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,furnature and interior,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19.0,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,audio and video,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9.0,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,goods for children and toys,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286.0,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,cars,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3.0,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,goods for children and toys,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9.0,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,repair and construction,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125.0,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,clothes shoes accessories,Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61.0,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,clothes shoes accessories,Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85.0,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,childrens clothing and footwear,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136.0,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0,0.00000


In [4]:
kmeans = KMeans(n_clusters=15).fit(vggFeat)

In [5]:
newDF = trainDF.assign( clusters = kmeans.predict(vggFeat) )

In [6]:
newDF.groupby("category_name").mean()

,price,item_seq_number,image_top_1,deal_probability,clusters
category_name,,,,,
apartment,2.546515e+06,6972.709406,1979.139283,0.141749,6.853019
appliances,6.586079e+03,924.109524,1875.843333,0.246594,7.003810
aquarium,2.147134e+03,107.926829,1909.533101,0.187944,7.195122
audio and video,6.030339e+03,772.950543,2835.417370,0.166314,6.916767
beauty and health,2.220361e+03,107.854266,1022.665607,0.085842,6.857446
bicycles,8.260895e+03,410.218935,2287.826923,0.228638,6.798817
birds,2.312667e+03,56.457249,2122.342007,0.238800,6.460967
books and mags,3.011519e+04,281.602961,2091.068641,0.029373,6.803499
cars,3.916758e+05,382.294012,1531.799642,0.288907,7.159161


In [9]:
newDF.groupby("category_name")["clusters"].agg(lambda x: stats.mode(x)[0][0]).sort_values()

category_name
tablets and ebooks                      2
ready business                          2
birds                                   2
propery abroad                          5
apartment                              12
games, consoles and programs           12
garages and parking places             12
goods for pets                         12
houses and cottages                    12
land                                   12
other animals                          12
office equipment and consumables       12
furnature and interior                 12
phones                                 12
photographic equipment                 12
plants                                 12
offer of services                      12
food                                   12
goods for children and toys            12
dogs                                   12
appliances                             12
aquarium                               12
audio and video                        12
beauty and health   

In [8]:
newDF.groupby("clusters")["deal_probability"].describe()

,count,mean,std,min,25%,50%,75%,max
clusters,,,,,,,,
0,4497.0,0.137773,0.264031,0.0,0.0,0.0,0.137180,1.0
1,7263.0,0.138850,0.263952,0.0,0.0,0.0,0.133530,1.0
2,8478.0,0.135316,0.260782,0.0,0.0,0.0,0.132370,1.0
3,2469.0,0.137848,0.262721,0.0,0.0,0.0,0.133010,1.0
4,8765.0,0.133027,0.258436,0.0,0.0,0.0,0.128690,1.0
5,7095.0,0.133324,0.258913,0.0,0.0,0.0,0.130250,1.0
6,7379.0,0.136821,0.262542,0.0,0.0,0.0,0.138920,1.0
7,7523.0,0.132124,0.257806,0.0,0.0,0.0,0.128690,1.0
8,5241.0,0.134480,0.259084,0.0,0.0,0.0,0.128690,1.0


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(np.vstack(trainDF["vgg"].as_matrix()))

In [ ]:
res = pca.transform(np.vstack(trainDF["vgg"].as_matrix()))

In [ ]:
for cat in trainDF["category_name"].unique():
    tmp = trainDF[trainDF["category_name"]==cat][0:4000]
    plt.figure(figsize=(10,10))
    for index, row in tmp.iterrows():
        res = pca.transform(np.array(row["vgg"]).reshape(1,4096))
        plt.scatter(res[0][0], res[0][1],c='r', alpha=row["deal_probability"])
        
    plt.show()

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(np.vstack(trainDF["vgg"][0:3000].as_matrix()))

In [ ]:
tmp = trainDF[0:3000]
plt.figure(figsize=(10,10))
for index, row in tmp.iterrows():
    plt.scatter(X_embedded[index][0], X_embedded[index][1],c='r', alpha=row["deal_probability"])

plt.show()

In [ ]:
plt.scatter(X_embedded[:,0],X_embedded[:,1])

In [ ]:
 X_embedded[999]